# Tool execution notebook
 this notebook  is use to make the experiments before  coed in the CI  process

In [ ]:

# export R2R_API_KEY=...
import os
from dotenv import load_dotenv
from r2r import R2RClient


load_dotenv()
# https://r2r-docs.sciphi.ai/documentation/search-and-rag
def search(query: str):
  
  user = os.getenv("R2R_USER")  
  pwd = os.getenv("R2R_PWD")  
  base_url = os.getenv("R2R_URL", "http://localhost:7272")
  client = R2RClient(base_url=base_url) 
  client.users.login(user, pwd)

  response = client.retrieval.rag(
    query=query
  )
  return response


In [ ]:
## test the calls to search the documents
response = search("explain  differences between leadin and leadout in expert")


In [ ]:
print(response.results.generated_answer)